## 4.2 - Stop right now, thank you very much


#### Aluna: Ana Luiza Poletto Loss 

**Objetivo**: implemente uma estratégia de Parada Antecipada (Early Stopping) no processo de treino da rede neural feita em Python puro ou no processo de treino da rede neural feita em PyTorch.

Comentário: esta não é para resolver com o módulo lightning.

### Introdução

As redes neurais, quando treinadas por muitas épocas, correm o risco de sofrer overfitting, ou seja, se ajustarem demais aos dados de treino, perdendo capacidade de generalização. Uma estratégia eficiente para diminuir esse problema é o uso de parada antecipada (Early Stopping). Ela interrompe o treinamento quando a performance na base de validação deixa de melhorar, evitando treinamento desnecessário e sobreajuste.

Este notebook tem como objetivo implementar uma rede neural simples usando `PyTorch`, treinada para prever a largura da pétala da espécie Iris virginica com base em outras características. Além disso, aplicamos a técnica de Early Stopping, monitorando a perda na base de validação.


##### Considerações

*O PyTorch é uma biblioteca de aprendizado de máquina muito utilizada para desenvolvimento de modelos de deep learning (O QUE É PYTORCH?, 2025). A técnica de Early Stopping permite interromper o treinamento da rede neural quando ela para de melhorar, evitando overfitting (CYBORG CODES, 2021). A implementação proposta segue a estrutura apresentada no notebook da disciplina (CASSAR, 2025).*

*Além disso, o texto colocado aqui foi revisado pelo modelo de linguagem.*

####  Desenvolvimento

Importamos bibliotecas essenciais: 

- `PyTorch` para construir e treinar a rede, `sklearn` para divisão dos dados e cálculo do erro, e `Seaborn/Matplotlib` para carregamento e visualização dos dados.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

Carregamos o `dataset` Iris que contém informações sobre flores, como comprimento e largura das sépalas e pétalas.

In [2]:
flores = sns.load_dataset('iris')
flores

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


Filtramos apenas a espécie virginica e selecionamos três características como entrada e uma como alvo o tamanho da pétala.

In [3]:
feature = ["sepal_length", "sepal_width", "petal_length"]
target = ["petal_width"]

filtro = flores["species"] == "virginica"
flores = flores.loc[filtro]

flores

,sepal_length,sepal_width,petal_length,petal_width,species
100,6.3,3.3,6.0,2.5,virginica
101,5.8,2.7,5.1,1.9,virginica
102,7.1,3.0,5.9,2.1,virginica
103,6.3,2.9,5.6,1.8,virginica
104,6.5,3.0,5.8,2.2,virginica
105,7.6,3.0,6.6,2.1,virginica
106,4.9,2.5,4.5,1.7,virginica
107,7.3,2.9,6.3,1.8,virginica
108,6.7,2.5,5.8,1.8,virginica
109,7.2,3.6,6.1,2.5,virginica


Organizamos os dados, retirando possíveis valores ausentes e separando em variáveis X (entradas) e y (alvo).

In [4]:
flores = flores.reindex(feature + target, axis=1)
flores = flores.dropna()

X = flores[feature].values
y = flores[target].values.reshape(-1,1)

flores

,sepal_length,sepal_width,petal_length,petal_width
100,6.3,3.3,6.0,2.5
101,5.8,2.7,5.1,1.9
102,7.1,3.0,5.9,2.1
103,6.3,2.9,5.6,1.8
104,6.5,3.0,5.8,2.2
105,7.6,3.0,6.6,2.1
106,4.9,2.5,4.5,1.7
107,7.3,2.9,6.3,1.8
108,6.7,2.5,5.8,1.8
109,7.2,3.6,6.1,2.5


Dividimos os dados em treino (20%), validação (20%) e teste (60%), garantindo aleatoriedade controlada com uma semente.

In [5]:
SEMENTE = 25

train_ratio = 0.2
validation_ratio = 0.2
test_ratio = 0.6

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=SEMENTE)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=SEMENTE)

Convertendo os dados `numpy` para tensores do `PyTorch`, que é o formato necessário para o treinamento da rede.

In [6]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
 
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
 
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

Criamos uma rede neural com duas camadas ocultas utilizando a classe `nn.Module` do PyTorch. As ativações são Sigmoid e a saída é linear.

In [7]:
class MLP(nn.Module):
    def __init__(self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets):
        super().__init__()
        
        self.camadas = nn.Sequential(
            nn.Linear(num_dados_entrada, neuronios_c1),
            nn.Sigmoid(),
            nn.Linear(neuronios_c1, neuronios_c2),
            nn.Sigmoid(),
            nn.Linear(neuronios_c2, num_targets),
        )
        
    def forward(self, x):
        x = self.camadas(x)
        return x

Instanciamos a rede com 3 entradas (features), 2 camadas ocultas e 1 saída (alvo).

In [8]:
NUM_DADOS_DE_ENTRADA = 3  
NUM_DADOS_DE_SAIDA = 1  
NEURONIOS_C1 = 3
NEURONIOS_C2 = 2

minha_mlp = MLP(
    NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA
)

Utilizamos o otimizador SGD (Gradiente Estocástico) e a função de perda MSELoss (erro quadrático médio).

In [9]:
TAXA_DE_APRENDIZADO = 0.001

otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)

fn_perda = nn.MSELoss()

Definimos os parâmetros do Early Stopping:

- PATIENCE: quantidade de épocas consecutivas sem melhora necessária para parar (10).

- DELTA: variação mínima aceitável para considerar uma melhora (0.01).

E então começamos o treinamento da rede com o Early Stopping. O loop executa as seguintes etapas:

*Treino:*

- Faz forward pass, calcula a perda e atualiza os pesos.

*Validação:*

- Calcula a perda na base de validação.

*Early Stopping:*

- Compara a perda atual da validação com a anterior.

- Se a melhora for menor que DELTA, incrementa um contador.

- Se isso acontecer por PATIENCE épocas consecutivas, o treinamento para.

In [11]:
NUM_EPOCAS = 100
PATIENCE = 10
DELTA = 0.01

perda_treino = []
perda_validacao = []
contador = 0


for epoca in range(NUM_EPOCAS):
    
    minha_mlp.train()
    # forward pass
    y_pred = minha_mlp(X_train)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_train, y_pred)
    perda_treino.append(loss)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado (opcional)
    print(epoca, loss.data)
    
    
    # VALIDAÇÃO 
    minha_mlp.eval()
    
    with torch.no_grad():
        # forward pass
        y_pred = minha_mlp(X_val)
 
        # calcula perda
        perda_validacao += [fn_perda(y_val, y_pred).data]
        
    
    # EARLY STOPPING
    penultimo_indice = len(perda_validacao) - 2
    penultimo_valor = perda_validacao[penultimo_indice]

    diferenca = abs(ultimo_valor - penultimo_valor)

    if diferenca <= DELTA:
        contador += 1
    else:
        contador = 0

    if contador == PATIENCE:
        parada = epoca
        print(f'Época de parada: {parada}')
        break

0 tensor(3.2025)
1 tensor(3.1820)
2 tensor(3.1616)
3 tensor(3.1413)
4 tensor(3.1212)
5 tensor(3.1013)
6 tensor(3.0814)
7 tensor(3.0618)
8 tensor(3.0422)
9 tensor(3.0228)
10 tensor(3.0036)
11 tensor(2.9844)
12 tensor(2.9654)
13 tensor(2.9466)
14 tensor(2.9278)
15 tensor(2.9092)
16 tensor(2.8908)
17 tensor(2.8724)
18 tensor(2.8542)
19 tensor(2.8361)
20 tensor(2.8182)
21 tensor(2.8003)
22 tensor(2.7826)
23 tensor(2.7650)
24 tensor(2.7476)
25 tensor(2.7302)
26 tensor(2.7130)
27 tensor(2.6959)
28 tensor(2.6789)
29 tensor(2.6620)
30 tensor(2.6453)
31 tensor(2.6286)
32 tensor(2.6121)
33 tensor(2.5957)
34 tensor(2.5794)
35 tensor(2.5632)
36 tensor(2.5472)
37 tensor(2.5312)
38 tensor(2.5153)
39 tensor(2.4996)
40 tensor(2.4840)
41 tensor(2.4684)
42 tensor(2.4530)
43 tensor(2.4377)
44 tensor(2.4225)
45 tensor(2.4074)
46 tensor(2.3924)
47 tensor(2.3774)
48 tensor(2.3626)
49 tensor(2.3479)
50 tensor(2.3333)
51 tensor(2.3188)
52 tensor(2.3044)
53 tensor(2.2901)
54 tensor(2.2759)
55 tensor(2.2618)
56

A rede é colocada em modo de avaliação e faz previsões sobre o conjunto de teste.

In [12]:
minha_mlp.eval()

with torch.no_grad():
    y_previsto = minha_mlp(X_test)
    
y_previsto

tensor([[0.7219],
        [0.7226],
        [0.7194],
        [0.7234],
        [0.7265],
        [0.7230],
        [0.7228],
        [0.7236],
        [0.7230],
        [0.7245],
        [0.7249],
        [0.7200],
        [0.7209],
        [0.7217],
        [0.7231],
        [0.7217],
        [0.7262],
        [0.7209],
        [0.7214],
        [0.7238],
        [0.7258],
        [0.7256],
        [0.7230],
        [0.7219],
        [0.7229],
        [0.7183],
        [0.7286],
        [0.7248],
        [0.7252],
        [0.7220]])

Calculamos o RMSE (Root Mean Squared Error), que mede o quão bem a rede conseguiu prever no conjunto de teste.

In [13]:
RMSE = mean_squared_error(y_test, y_previsto, squared=False)
print(RMSE)

1.3013108


### Conclusão

O experimento obteve sucesso na implementação de uma rede neural simples com o PyTorch e na aplicação da estratégia de Early Stopping. Observou-se uma redução contínua da perda durante o treinamento, indicando que o modelo conseguiu aprender os padrões dos dados.

O RMSE de aproximadamente 1.30 reflete um erro razoável, dado o tamanho pequeno do dataset e a simplicidade da rede. A técnica de Early Stopping, apesar de não ter sido ativada neste caso, foi corretamente implementada, pronta para interromper o treinamento caso a melhora na validação estagnasse, o que é uma estratégia eficaz para evitar overfitting em modelos mais complexos ou datasets maiores.

### Referências


O QUE É PYTORCH? IBM. Disponível em: https://www.ibm.com/br-pt/think/topics/pytorch. Acesso em: 12 jun. 2025.

CASSAR, Daniel Roberto. ATP-303 NN 5.2 — Notebook PyTorch. 2025.

CYBORG CODES. What is Early Stopping in Deep Learning? Medium. Disponível em: https://cyborgcodes.medium.com/what-is-early-stopping-in-deep-learning-eeb1e710a3cf. Acesso em: 13 jun. 2025.